<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e307c935308ec0edbcadd402d2e6639befcba517754906fc081bf384dca87952
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-19 13:45:44
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -7.89 K (-0.05%)
Current PnL: -21.06 L (-13.8%)
CY Booked + Current PnL: -7.10 L (-4.65%)
-------------------
Total profit:  1.81 L
Total loss:  -22.88 L
-------------------
Total Booked + Current PnL: 19.88 L (15.82%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.64%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 90.41 L (62.44%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,146.65,79.0,M-SC,1.89,161820.0,-12739.0,12768.0,-1.58,-7.30,7.89,0.02,234.0,-1.00,1.12,40.76,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.73,86650.0,-14127.0,14219.0,0.23,-14.02,16.41,0.09,253.0,-0.99,0.60,13.84,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.85,88620.0,-2058.0,16962.0,-0.42,-2.27,19.14,16.44,63.0,-0.12,0.61,17.45,X40,ATH,PHARMA
35,ICICIGI,2252.93,-14.60,59.0,X-MC,7.60,190157.0,15837.0,19358.0,0.93,9.08,10.18,20.19,68.0,0.82,1.31,25.57,X40,ATH,INSURANCE
84,VOLTAS,1530.00,-1.22,56.0,X-MC,3.72,209985.0,18243.0,19508.0,1.13,9.51,9.29,19.69,78.0,0.94,1.45,16.96,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7201.88,5.43,68.0,H-LC,11.48,270248.0,24097.0,53833.0,4.33,9.79,19.92,31.66,43.0,0.45,1.87,49.95,X200,ATH,IT
32,HCLTECH,1922.01,12.76,77.0,X-LC,4.38,260934.0,19010.0,40810.0,4.19,7.86,15.64,24.73,7.0,0.47,1.80,24.29,X40,ATH,IT
41,INFY,2275.00,-18.52,59.0,X-LC,3.63,326848.0,13960.0,157737.0,3.24,4.46,48.26,54.87,2.0,0.09,2.26,13.11,X40,BTT,IT
13,BSOFT,831.70,-9.16,61.0,H-SC,8.21,99807.0,-40846.0,112273.0,3.04,-29.04,112.49,50.79,151.0,-0.36,0.69,16.21,XR,ATH,IT
69,SONACOMS,806.63,-31.96,68.0,M-SC,6.76,87972.0,-13187.0,53188.0,2.90,-13.04,60.46,39.54,220.0,-0.25,0.61,24.05,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,104.33,59.0,H-SC,16.04,138861.0,-79635.0,107826.0,-3.71,-36.45,77.65,12.90,161.0,-0.74,0.96,15.46,OX40N,NTT,FINANCE
63,SAMMAANCAP,326.00,18.70,52.0,M-SC,36.55,159588.0,10368.0,133815.0,-2.98,6.95,83.85,96.62,245.0,0.08,1.10,72.71,XY25,NTT,FINANCE
77,TMPV,1065.00,-23.08,21.0,X-LC,0.23,237044.0,-39366.0,460529.0,-2.53,-14.24,194.28,152.37,3.0,-0.09,1.64,0.00,XY24,NTT,AUTO
36,ICICIPRULI,790.00,-22.52,50.0,X-MC,1.41,179155.0,2417.0,52313.0,-2.45,1.37,29.20,30.97,75.0,0.05,1.24,14.10,X40,ATH,INSURANCE
51,MASFIN,398.61,-18.25,51.0,H-SC,7.17,92940.0,-5040.0,26646.0,-2.36,-5.14,28.67,22.05,164.0,-0.19,0.64,35.10,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-20.55,40.0,M-SC,10.15,144177.0,-3467.0,162790.0,-1.80,-2.35,112.91,107.91,216.0,-0.02,1.00,0.58,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,146.65,79.0,M-SC,1.89,161820.0,-12739.0,12768.0,-1.58,-7.30,7.89,0.02,234.0,-1.00,1.12,40.76,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.73,86650.0,-14127.0,14219.0,0.23,-14.02,16.41,0.09,253.0,-0.99,0.60,13.84,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.37,43.0,H-SC,1.99,220554.0,-49113.0,85443.0,-0.77,-18.21,38.74,13.47,158.0,-0.57,1.52,12.15,XY24,NTT,PAINTS
74,TATAELXSI,9161.00,-16.37,49.0,H-SC,6.34,101906.0,-29929.0,72149.0,1.64,-22.70,70.80,32.03,148.0,-0.41,0.70,14.14,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,18.70,52.0,M-SC,36.55,159588.0,10368.0,133815.0,-2.98,6.95,83.85,96.62,245.0,0.08,1.10,72.71,XY25,NTT,FINANCE
86,WIPRO,420.00,-13.34,55.0,M-LC,5.13,152452.0,1507.0,107951.0,2.07,1.00,70.81,72.51,99.0,0.01,1.05,7.08,XR,NTT,IT
38,INDIAMART,4810.62,-48.72,44.0,H-SC,4.57,127868.0,4532.0,127101.0,-1.37,3.67,99.40,106.72,139.0,0.04,0.88,27.91,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-48.72,44.0,H-SC,4.57,127868.0,4532.0,127101.0,-1.37,3.67,99.40,106.72,139.0,0.04,0.88,27.91,AR,ATH,MISC
86,WIPRO,420.00,-13.34,55.0,M-LC,5.13,152452.0,1507.0,107951.0,2.07,1.00,70.81,72.51,99.0,0.01,1.05,7.08,XR,NTT,IT
25,FINCABLES,1641.55,-20.55,40.0,M-SC,10.15,144177.0,-3467.0,162790.0,-1.80,-2.35,112.91,107.91,216.0,-0.02,1.00,0.58,OX40N,ATH,CABLES
85,WHIRLPOOL,2270.00,-43.39,35.0,M-SC,7.83,110952.0,-3846.0,97893.0,-0.07,-3.35,88.23,81.92,236.0,-0.04,0.77,31.35,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.85,34.0,H-MC,5.24,205784.0,-3790.0,78465.0,0.30,-1.81,38.13,35.63,119.0,-0.05,1.42,14.56,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-23.08,21.0,X-LC,0.23,237044.0,-39366.0,460529.0,-2.53,-14.24,194.28,152.37,3.0,-0.09,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.90,83813.0,-44857.0,90158.0,-0.49,-34.86,107.57,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-26.58,28.0,X-SC,1.31,104385.0,-23553.0,70690.0,-0.10,-18.41,67.72,36.84,90.0,-0.33,0.72,6.86,X40N,ATH,ELECTRICAL
26,GILLETTE,11206.78,-19.05,29.0,X-SC,6.11,249735.0,-4161.0,86458.0,-0.23,-1.64,34.62,32.42,85.0,-0.05,1.73,13.21,X40,ATH,FMCG
46,JSWINFRA,342.00,-20.15,30.0,X-MC,5.97,187076.0,-13271.0,43776.0,0.36,-6.62,23.40,15.23,66.0,-0.30,1.29,23.49,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-23.08,21.0,X-LC,0.23,237044.0,-39366.0,460529.0,-2.53,-14.24,194.28,152.37,3.0,-0.09,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.00,-41.15,43.0,X-LC,0.91,197752.0,-2386.0,23275.0,-0.36,-1.19,11.77,10.44,5.0,-0.10,1.37,4.30,X40,NTT,FMCG
34,HONAUT,58357.33,-26.58,28.0,X-SC,1.31,104385.0,-23553.0,70690.0,-0.10,-18.41,67.72,36.84,90.0,-0.33,0.72,6.86,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-22.52,50.0,X-MC,1.41,179155.0,2417.0,52313.0,-2.45,1.37,29.20,30.97,75.0,0.05,1.24,14.10,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-0.97,61.0,H-LC,1.78,161450.0,-24645.0,72120.0,-0.63,-13.24,44.67,25.51,49.0,-0.34,1.12,18.98,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.90,83813.0,-44857.0,90158.0,-0.49,-34.86,107.57,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-30.18,33.0,X-MC,8.19,155240.0,-8500.0,51183.0,-0.82,-5.19,32.97,26.07,55.0,-0.17,1.07,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-23.08,21.0,X-LC,0.23,237044.0,-39366.0,460529.0,-2.53,-14.24,194.28,152.37,3.0,-0.09,1.64,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-19.44,30.0,X-MC,6.93,173988.0,-26518.0,123671.0,-0.46,-13.23,71.08,48.45,64.0,-0.21,1.20,0.33,X40N,ATH,FINANCE
56,QUESS,424.00,-53.17,34.0,X-SC,37.22,46647.0,-18359.0,46209.0,1.07,-28.24,99.06,42.84,83.0,-0.40,0.32,1.07,XY24,NTT,MISC


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-53.17,34.0,X-SC,37.22,46647.0,-18359.0,46209.0,1.07,-28.24,99.06,42.84,83.0,-0.40,0.32,1.07,XY24,NTT,MISC
60,RELAXO,1176.00,-42.79,33.0,X-SC,3.14,78020.0,-67140.0,143065.0,-1.52,-46.25,183.37,52.31,91.0,-0.47,0.54,4.43,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.90,83813.0,-44857.0,90158.0,-0.49,-34.86,107.57,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.85,88620.0,-2058.0,16962.0,-0.42,-2.27,19.14,16.44,63.0,-0.12,0.61,17.45,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.58,28.0,X-SC,1.31,104385.0,-23553.0,70690.0,-0.10,-18.41,67.72,36.84,90.0,-0.33,0.72,6.86,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-25.57,63.0,X-LC,8.12,295517.0,-50439.0,117143.0,1.84,-14.58,39.64,19.28,1.0,-0.43,2.04,9.25,X40,ATH,IT
41,INFY,2275.00,-18.52,59.0,X-LC,3.63,326848.0,13960.0,157737.0,3.24,4.46,48.26,54.87,2.0,0.09,2.26,13.11,X40,BTT,IT
77,TMPV,1065.00,-23.08,21.0,X-LC,0.23,237044.0,-39366.0,460529.0,-2.53,-14.24,194.28,152.37,3.0,-0.09,1.64,0.00,XY24,NTT,AUTO
83,VBL,671.64,-18.42,47.0,X-LC,6.03,292140.0,-23702.0,136371.0,-0.82,-7.50,46.68,35.67,4.0,-0.17,2.02,5.31,X40N,ATH,FMCG
43,ITC,452.00,-41.15,43.0,X-LC,0.91,197752.0,-2386.0,23275.0,-0.36,-1.19,11.77,10.44,5.0,-0.10,1.37,4.30,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7561.11,60.0,L-SC,18.00,84032.0,-9778.0,86393.0,-2.04,-10.42,102.81,81.67,271.0,-0.11,0.58,62.73,XR,NTT,CERAMICS
69,SONACOMS,806.63,-31.96,68.0,M-SC,6.76,87972.0,-13187.0,53188.0,2.90,-13.04,60.46,39.54,220.0,-0.25,0.61,24.05,AR,ATH,AUTO
51,MASFIN,398.61,-18.25,51.0,H-SC,7.17,92940.0,-5040.0,26646.0,-2.36,-5.14,28.67,22.05,164.0,-0.19,0.64,35.10,XR,ATH,FINANCE
71,SURYODAY,216.00,53.51,45.0,H-SC,12.65,141729.0,-37342.0,78589.0,0.59,-20.85,55.45,23.03,167.0,-0.48,0.98,40.62,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,63.64,49.0,H-SC,10.76,141733.0,-41042.0,152108.0,-1.95,-22.46,107.32,60.77,159.0,-0.27,0.98,27.64,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,146.65,79.0,M-SC,1.89,161820.0,-12739.0,12768.0,-1.58,-7.30,7.89,0.02,234.0,-1.00,1.12,40.76,OX40N,NTT,PAINTS
13,BSOFT,831.70,-9.16,61.0,H-SC,8.21,99807.0,-40846.0,112273.0,3.04,-29.04,112.49,50.79,151.0,-0.36,0.69,16.21,XR,ATH,IT
5,ASIANPAINT,3460.25,0.35,83.0,X-LC,10.86,255200.0,3432.0,49305.0,-0.21,1.36,19.32,20.95,31.0,0.07,1.76,37.49,X40,ATH,PAINTS
81,VAIBHAVGBL,521.00,63.64,49.0,H-SC,10.76,141733.0,-41042.0,152108.0,-1.95,-22.46,107.32,60.77,159.0,-0.27,0.98,27.64,XR,NTT,JEWELLERY
32,HCLTECH,1922.01,12.76,77.0,X-LC,4.38,260934.0,19010.0,40810.0,4.19,7.86,15.64,24.73,7.0,0.47,1.80,24.29,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.69
1,25,44.50
2,50,75.17


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.51
MC    28.63
LC    24.85
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.88
X40      20.59
X40N     11.50
XY25     10.14
XR        9.49
AR        9.05
OX40N     7.66
X200      1.87
SR        1.00
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.19
X-MC    22.25
X-LC    19.84
M-SC    13.20
X-SC     7.71
H-MC     4.69
H-LC     2.99
L-SC     1.41
M-MC     1.36
M-LC     1.05
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.66,-4.42,38.19
IT,13.60,-13.07,71.49
FINANCE,10.45,-12.26,65.05
MISC,7.36,-21.75,71.62
PAINTS,6.05,-5.27,20.91
ELECTRICAL,5.62,-11.20,50.62
INSURANCE,4.47,2.29,32.14
PHARMA,3.41,-0.94,34.81
AUTO,3.33,-19.26,133.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3280585.0,21
AR,1262179.0,10
XR,1258680.0,13
X40,987164.0,14
X40N,705960.0,9
OX40N,683657.0,10
XY25,465731.0,7
SR,275529.0,2
MH,67487.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3476303.0,25
M-SC,1441995.0,16
X-LC,1195346.0,11
X-MC,1170330.0,16
X-SC,724772.0,8
H-MC,399858.0,3
L-SC,257249.0,3
H-LC,125953.0,2
M-LC,107951.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1181333.0      6
           AR         851892.0      5
           XR         780456.0      7
M-SC       XY24       763549.0      6
X-LC       XY24       531143.0      2
X-MC       X40        422278.0      7
X-LC       X40        388270.0      5
X-MC       XY24       341191.0      3
H-SC       OX40N      319606.0      4
X-SC       XY24       279577.0      3
M-SC       OX40N      276524.0      5
H-SC       SR         275529.0      2
X-SC       X40N       268579.0      3
X-MC       X40N       232579.0      4
H-MC       AR         216066.0      2
X-LC       X40N       204802.0      2
H-MC       XY24       183792.0      1
X-SC       X40        176616.0      2
X-MC       XY25       174282.0      2
L-SC       XR         169722.0      2
M-SC       XR         146006.0      2
           XY25       133815.0      1
           AR         122101.0      2
M-LC       XR         107951.0      1
L-SC       OX40N       87527.0      1
H-LC       AR          72120.0      1
X-LC       XY25        71131.0      2
H-SC       MH          67487.0      1
L-MC       XR          54545.0      1
H-LC       X200        53833.0      1
M-MC       XY25        48065.0      1
L-LC       XY25        38438.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
